TODO :
Implémentation
Factory pour wake word et STT
Dans les configs : pouvoir configurer le micro, les keywords, models wake word, model STT, le temps d'écoute/enregistrement

In [2]:
# %sudo apt-get update
# %sudo apt-get install python3-pip python3-dev portaudio19-dev
%pip install sounddevice vosk pvporcupine numpy


   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/13.1 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/13.1 MB 7.2 MB/s eta 0:00:02
   ------------ --------------------------- 3.9/13.1 MB 6.9 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/13.1 MB 6.8 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/13.1 MB 6.7 MB/s eta 0:00:01
   ------------------------ --------------- 8.1/13.1 MB 6.7 MB/s eta 0:00:01
   ----------------------------- ---------- 9.7/13.1 MB 6.9 MB/s eta 0:00:01
   --------------------------------- ------ 11.0/13.1 MB 6.7 MB/s eta 0:00:01
   ------------------------------------- -- 12.3/13.1 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 6.7 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %wget https://alphacephei.com/vosk/models/vosk-model-small-fr-0.22.zip
# %unzip vosk-model-small-fr-0.22.zip


'wget' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.
'unzip' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [7]:
import sounddevice as sd

print(sd.query_devices())  # Liste des périphériques
print("Default device:", sd.default.device)

# Test simple enregistrement + relecture
samplerate = 16000
print("Enregistrement 3s...")
rec = sd.rec(int(3 * samplerate), samplerate=samplerate, channels=1, dtype='float32')
sd.wait()
print("Lecture...")
sd.play(rec, samplerate)
sd.wait()


   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Réseau de microphones (Technolo, MME (4 in, 0 out)
   2 Microphone (WO Mic Device), MME (1 in, 0 out)
   3 Microphone (Steam Streaming Mic, MME (8 in, 0 out)
   4 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  5 SONY TV (HD Audio Driver for Di, MME (0 in, 2 out)
   6 Haut-parleurs (Realtek(R) Audio, MME (0 in, 2 out)
   7 Haut-parleurs (Steam Streaming , MME (0 in, 8 out)
   8 Haut-parleurs (Steam Streaming , MME (0 in, 8 out)
   9 Pilote de capture audio principal, Windows DirectSound (2 in, 0 out)
  10 Réseau de microphones (Technologie Intel® Smart Sound pour microphones numériques), Windows DirectSound (4 in, 0 out)
  11 Microphone (WO Mic Device), Windows DirectSound (1 in, 0 out)
  12 Microphone (Steam Streaming Microphone), Windows DirectSound (8 in, 0 out)
  13 Périphérique audio principal, Windows DirectSound (0 in, 2 out)
  14 SONY TV (HD Audio Driver for Display Audio), Windows DirectSound (0 in, 2 out)
  15 

In [2]:
import sounddevice as sd
import queue
import json
import numpy as np
import pvporcupine
from vosk import Model, KaldiRecognizer

# ==== File audio ====
audio_q = queue.Queue()

def audio_callback(indata, frames, time, status):
    if status:
        print(status)
    audio_q.put(indata.copy())

# ==== Init Porcupine (wake word) ====
porcupine = pvporcupine.create(
    access_key="YTZtqOjD4eEEySY1rj2a3+cn2bNg38bJxzET+jn3SE5jpF9eiTx3hA==",
    model_path="wake_word_model/porcupine_params_de.pv",
    keyword_paths=["wake_words/glados_de_windows_v3_0_0.ppn"],
    # keywords=["jarvis"]
)  # Wake word "Jarvis"

# ==== Init Vosk (STT) ====
model = Model("vosk-model-small-fr-0.22")
recognizer = KaldiRecognizer(model, 16000)

print("Assistant en écoute... (dites 'Jarvis' pour activer)")

with sd.InputStream(channels=1, samplerate=16000, callback=audio_callback, blocksize=512):
    pcm_buffer = []

    while True:
        audio = audio_q.get()
        pcm = (audio.flatten() * 32767).astype(np.int16)

        # Découpage en frames pour Porcupine
        for i in range(0, len(pcm), porcupine.frame_length):
            frame = pcm[i:i+porcupine.frame_length]
            if len(frame) == porcupine.frame_length:
                keyword_index = porcupine.process(frame)
                if keyword_index >= 0:
                    print("Wake word détecté ! Parlez maintenant...")

                    # Enregistrer quelques secondes
                    rec = sd.rec(int(3 * 16000), samplerate=16000, channels=1, dtype='int16')
                    sd.wait()

                    # Reconnaissance Vosk
                    if recognizer.AcceptWaveform(rec.tobytes()):
                        result = json.loads(recognizer.Result())
                        print("Texte:", result.get("text", ""))
                    else:
                        partial = json.loads(recognizer.PartialResult())
                        print("Texte (partiel):", partial.get("partial", ""))


Assistant en écoute... (dites 'Jarvis' pour activer)
Wake word détecté ! Parlez maintenant...
Texte: allume la lumière
Wake word détecté ! Parlez maintenant...
Texte (partiel): tu vas éteindre la lumière et faire


KeyboardInterrupt: 

In [ ]:
# Variante : détection de fin de phrase (silence) pour capter la phrase entière sans doublon
import time
def detect_silence(audio, threshold=500):
    # audio: numpy array int16
    # Retourne True si silence détecté
    energy = np.abs(audio).mean()
    return energy < threshold

print("Assistant en écoute... (dites 'Jarvis' pour activer)")
min_silence_len = 1  # durée minimale de silence (en secondes) pour arrêter l'écoute
with sd.InputStream(channels=1, samplerate=16000, callback=audio_callback, blocksize=512):
    listening = False
    while True:
        audio = audio_q.get()
        pcm = (audio.flatten() * 32767).astype(np.int16)
        if not listening:
            for i in range(0, len(pcm), porcupine.frame_length):
                frame = pcm[i:i+porcupine.frame_length]
                if len(frame) == porcupine.frame_length:
                    keyword_index = porcupine.process(frame)
                    if keyword_index >= 0:
                        print("Wake word détecté ! Parlez maintenant...")
                        listening = True
                        frames = []
                        start_time = time.time()
                        max_duration = 8  # secondes max d'écoute
                        silence_count = 0
                        silence_frames = int((min_silence_len * 16000) // 512)
                        break
        if listening:
            # On écoute jusqu'à la fin de phrase (silence ou durée max)
            while True:
                audio = audio_q.get()
                frames.append(audio)
                pcm = (audio.flatten() * 32767).astype(np.int16)
                recognizer.AcceptWaveform(pcm.tobytes())
                if detect_silence(pcm):
                    silence_count += 1
                else:
                    silence_count = 0
                if silence_count >= silence_frames or (time.time() - start_time > max_duration):
                    break
            result = json.loads(recognizer.Result())
            print("Texte:", result.get("text", ""))
            listening = False

Assistant en écoute... (dites 'Jarvis' pour activer)
Wake word détecté ! Parlez maintenant...
Wake word détecté ! Parlez maintenant...
Texte: allume la lumière
Texte: allume la lumière
Wake word détecté ! Parlez maintenant...
Wake word détecté ! Parlez maintenant...
Texte: au faite
Texte: au faite
Wake word détecté ! Parlez maintenant...
Wake word détecté ! Parlez maintenant...
Texte: allume la lumière
Texte: allume la lumière
Wake word détecté ! Parlez maintenant...
Wake word détecté ! Parlez maintenant...
Texte: faites un coup
Texte: faites un coup
Wake word détecté ! Parlez maintenant...
Wake word détecté ! Parlez maintenant...
Texte: allume la lumière
Texte: allume la lumière
Wake word détecté ! Parlez maintenant...
Wake word détecté ! Parlez maintenant...
Texte: pete un coup
Texte: pete un coup


KeyboardInterrupt: 